## Vstavané reprezentácie

V našom predchádzajúcom príklade sme pracovali s vysokodimenzionálnymi vektormi bag-of-words s dĺžkou `vocab_size` a explicitne sme prevádzali nízkodimenzionálne pozičné reprezentácie na riedke one-hot reprezentácie. Táto one-hot reprezentácia nie je pamäťovo efektívna. Navyše, každé slovo je spracované nezávisle od ostatných, takže one-hot kódované vektory nevyjadrujú sémantické podobnosti medzi slovami.

V tejto jednotke budeme pokračovať v skúmaní datasetu **News AG**. Na začiatok načítajme dáta a získajme niektoré definície z predchádzajúcej jednotky.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Čo je embedding?

Myšlienka **embeddingu** spočíva v reprezentácii slov pomocou nízkodimenzionálnych hustých vektorov, ktoré odrážajú sémantický význam slova. Neskôr si povieme, ako vytvoriť zmysluplné word embeddings, ale zatiaľ si embedding predstavme ako spôsob zníženia dimenzionality vektora slova.

Embedding vrstva teda prijíma slovo ako vstup a produkuje výstupný vektor so špecifikovanou veľkosťou `embedding_size`. V istom zmysle je veľmi podobná vrstve `Dense`, ale namiesto toho, aby prijímala vektor zakódovaný pomocou one-hot, dokáže prijať číslo reprezentujúce slovo.

Použitím embedding vrstvy ako prvej vrstvy v našej sieti môžeme prejsť z modelu bag-of-words na model **embedding bag**, kde najskôr každé slovo v našom texte prevedieme na zodpovedajúci embedding a potom vypočítame nejakú agregačnú funkciu nad všetkými týmito embeddingmi, ako napríklad `sum`, `average` alebo `max`.

![Obrázok zobrazujúci embedding klasifikátor pre päť sekvenčných slov.](../../../../../translated_images/sk/embedding-classifier-example.b77f021a7ee67eee.webp)

Naša klasifikačná neurónová sieť pozostáva z nasledujúcich vrstiev:

* Vrstva `TextVectorization`, ktorá prijíma reťazec ako vstup a produkuje tenzor čísel tokenov. Špecifikujeme rozumnú veľkosť slovníka `vocab_size` a ignorujeme menej často používané slová. Vstupný tvar bude 1 a výstupný tvar bude $n$, pretože dostaneme $n$ tokenov ako výsledok, pričom každý z nich obsahuje čísla od 0 po `vocab_size`.
* Vrstva `Embedding`, ktorá prijíma $n$ čísel a redukuje každé číslo na hustý vektor danej dĺžky (v našom príklade 100). Tým sa vstupný tenzor tvaru $n$ transformuje na tenzor $n\times 100$.
* Agregačná vrstva, ktorá vypočíta priemer tohto tenzora pozdĺž prvej osi, t.j. vypočíta priemer všetkých $n$ vstupných tenzorov zodpovedajúcich rôznym slovám. Na implementáciu tejto vrstvy použijeme vrstvu `Lambda` a do nej odovzdáme funkciu na výpočet priemeru. Výstup bude mať tvar 100 a bude predstavovať číselnú reprezentáciu celého vstupného sekvenčného textu.
* Záverečný lineárny klasifikátor `Dense`.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


V `summary` výpise, v stĺpci **output shape**, prvý rozmer tenzora `None` zodpovedá veľkosti minibatchu a druhý zodpovedá dĺžke sekvencie tokenov. Všetky sekvencie tokenov v minibatchu majú rôzne dĺžky. O tom, ako s tým pracovať, budeme hovoriť v nasledujúcej sekcii.

Teraz poďme trénovať sieť:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Poznámka** že vytvárame vektorizačný nástroj na základe podmnožiny údajov. Toto sa robí s cieľom urýchliť proces, čo môže viesť k situácii, keď nie všetky tokeny z nášho textu budú prítomné vo slovníku. V takom prípade budú tieto tokeny ignorované, čo môže viesť k mierne nižšej presnosti. Avšak v reálnom živote podmnožina textu často poskytuje dobrý odhad slovníka.


### Práca s rôznymi veľkosťami sekvencií premenných

Poďme si vysvetliť, ako prebieha trénovanie v minibatchoch. V uvedenom príklade má vstupný tenzor rozmer 1 a používame minibatche s dĺžkou 128, takže skutočná veľkosť tenzora je $128 \times 1$. Počet tokenov v každej vete je však odlišný. Ak aplikujeme vrstvu `TextVectorization` na jeden vstup, počet vrátených tokenov sa líši v závislosti od toho, ako je text tokenizovaný:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Avšak, keď aplikujeme vektorizátor na niekoľko sekvencií, musí vytvoriť tenzor obdĺžnikového tvaru, takže nevyužité prvky vyplní tokenom PAD (ktorý je v našom prípade nula):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Tu môžeme vidieť vkladania:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Poznámka**: Na minimalizáciu množstva doplňovania má v niektorých prípadoch zmysel zoradiť všetky sekvencie v dátovej sade podľa rastúcej dĺžky (alebo presnejšie podľa počtu tokenov). Tým sa zabezpečí, že každý minibatch bude obsahovať sekvencie podobnej dĺžky.


## Sémantické vektory: Word2Vec

V našom predchádzajúcom príklade sa vrstva embedding naučila mapovať slová na vektorové reprezentácie, avšak tieto reprezentácie nemali sémantický význam. Bolo by užitočné naučiť sa vektorovú reprezentáciu tak, aby podobné slová alebo synonymá zodpovedali vektorom, ktoré sú blízko seba z hľadiska nejakej vektorovej vzdialenosti (napríklad euklidovskej vzdialenosti).

Aby sme to dosiahli, musíme predtrénovať náš embedding model na veľkej zbierke textov pomocou techniky, ako je [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Táto technika je založená na dvoch hlavných architektúrach, ktoré sa používajú na vytvorenie distribuovanej reprezentácie slov:

 - **Kontinuálny model bag-of-words** (CBoW), kde trénujeme model na predpovedanie slova na základe okolitého kontextu. Pri danom n-grame $(W_{-2},W_{-1},W_0,W_1,W_2)$ je cieľom modelu predpovedať $W_0$ na základe $(W_{-2},W_{-1},W_1,W_2)$.
 - **Kontinuálny skip-gram** je opačný k CBoW. Model používa okolité slová z kontextového okna na predpovedanie aktuálneho slova.

CBoW je rýchlejší, zatiaľ čo skip-gram je pomalší, ale lepšie reprezentuje zriedkavé slová.

![Obrázok zobrazujúci algoritmy CBoW a Skip-Gram na konverziu slov na vektory.](../../../../../translated_images/sk/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Na experimentovanie s embeddingom Word2Vec predtrénovaným na Google News dataset môžeme použiť knižnicu **gensim**. Nižšie nájdeme slová najviac podobné slovu 'neural'.

> **Poznámka:** Keď prvýkrát vytvárate vektorové reprezentácie slov, ich sťahovanie môže chvíľu trvať!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Môžeme tiež extrahovať vektorové zakotvenie zo slova, ktoré sa použije pri trénovaní klasifikačného modelu. Zakotvenie má 300 komponentov, ale tu ukazujeme iba prvých 20 komponentov vektora pre prehľadnosť:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Skvelá vec na semantických vkladoch je, že môžete manipulovať s vektorovým kódovaním na základe semantiky. Napríklad môžeme požiadať o nájdenie slova, ktorého vektorová reprezentácia je čo najbližšie k slovám *kráľ* a *žena*, a čo najďalej od slova *muž*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Príklad vyššie používa určitú internú mágiu GenSym, ale základná logika je v skutočnosti celkom jednoduchá. Zaujímavou vecou na vkladaní je, že môžete vykonávať bežné vektorové operácie na vektoroch vkladania, a to by odrážalo operácie na **významoch** slov. Príklad vyššie možno vyjadriť pomocou vektorových operácií: vypočítame vektor zodpovedajúci **KRÁĽ-MUŽ+ŽENA** (operácie `+` a `-` sa vykonávajú na vektorových reprezentáciách príslušných slov) a potom nájdeme najbližšie slovo v slovníku k tomuto vektoru:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: Museli sme pridať malé koeficienty k vektorom *muž* a *žena* - skúste ich odstrániť, aby ste videli, čo sa stane.

Na nájdenie najbližšieho vektora používame TensorFlow mechanizmus na výpočet vektora vzdialeností medzi naším vektorom a všetkými vektormi vo slovníku, a potom nájdeme index minimálneho slova pomocou `argmin`.


Aj keď sa Word2Vec javí ako skvelý spôsob na vyjadrenie sémantiky slov, má mnoho nevýhod, vrátane nasledujúcich:

* Modely CBoW a skip-gram sú **prediktívne vektorizácie**, ktoré berú do úvahy iba lokálny kontext. Word2Vec nevyužíva globálny kontext.
* Word2Vec neberie do úvahy **morfologickú štruktúru** slov, t. j. skutočnosť, že význam slova môže závisieť od rôznych častí slova, ako je napríklad koreň.

**FastText** sa snaží prekonať druhé obmedzenie a stavia na Word2Vec tým, že sa učí vektorové reprezentácie pre každé slovo a n-gramy znakov nachádzajúce sa v rámci každého slova. Hodnoty týchto reprezentácií sa potom pri každom kroku trénovania spriemerujú do jedného vektora. Aj keď to pridáva veľa dodatočných výpočtov počas predtrénovania, umožňuje to vektorovým reprezentáciám slov zakódovať informácie o podslovách.

Ďalšia metóda, **GloVe**, používa odlišný prístup k vektorovým reprezentáciám slov, založený na faktorizácii matice slov a kontextov. Najprv vytvorí veľkú maticu, ktorá počíta počet výskytov slov v rôznych kontextoch, a potom sa snaží túto maticu reprezentovať v nižších dimenziách tak, aby minimalizovala stratu pri rekonštrukcii.

Knižnica gensim podporuje tieto vektorové reprezentácie slov a môžete s nimi experimentovať zmenou kódu na načítanie modelu vyššie.


## Použitie predtrénovaných embeddingov v Keras

Môžeme upraviť vyššie uvedený príklad tak, aby sme predvyplnili maticu v našej embedding vrstve pomocou semantických embeddingov, ako je Word2Vec. Slovníky predtrénovaného embeddingu a textového korpusu sa pravdepodobne nebudú zhodovať, takže si musíme vybrať jeden. Tu skúmame dve možné možnosti: použitie slovníka tokenizéra a použitie slovníka z embeddingov Word2Vec.

### Použitie slovníka tokenizéra

Pri použití slovníka tokenizéra budú niektoré slová zo slovníka mať zodpovedajúce embeddingy Word2Vec, zatiaľ čo niektoré budú chýbať. Keďže veľkosť nášho slovníka je `vocab_size` a dĺžka embeddingového vektora Word2Vec je `embed_size`, embedding vrstva bude reprezentovaná váhovou maticou tvaru `vocab_size`$\times$`embed_size`. Túto maticu naplníme prechádzaním slovníka:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Pre slová, ktoré nie sú prítomné vo Word2Vec slovníku, môžeme buď ponechať hodnoty ako nuly, alebo vygenerovať náhodný vektor.

Teraz môžeme definovať vrstvu embedding s predtrénovanými váhami:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Poznámka**: Všimnite si, že sme nastavili `trainable=False` pri vytváraní `Embedding`, čo znamená, že vrstvu Embedding nebudeme znovu trénovať. To môže spôsobiť mierne nižšiu presnosť, ale urýchľuje to tréning.

### Použitie slovníka pre embedding

Jedným z problémov predchádzajúceho prístupu je, že slovníky použité v TextVectorization a Embedding sú odlišné. Na prekonanie tohto problému môžeme použiť jedno z nasledujúcich riešení:
* Znovu natrénovať Word2Vec model na našom slovníku.
* Načítať náš dataset so slovníkom z predtrénovaného Word2Vec modelu. Slovníky použité na načítanie datasetu môžeme špecifikovať počas načítania.

Druhý prístup sa zdá byť jednoduchší, takže ho implementujeme. Najprv vytvoríme vrstvu `TextVectorization` so špecifikovaným slovníkom, prevzatým z Word2Vec embeddingov:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Knižnica word embeddings Gensim obsahuje praktickú funkciu `get_keras_embeddings`, ktorá automaticky vytvorí zodpovedajúcu vrstvu embeddings pre Keras.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Jedným z dôvodov, prečo nevidíme vyššiu presnosť, je, že niektoré slová z nášho datasetu chýbajú v predtrénovanej slovnej zásobe GloVe, a preto sú v podstate ignorované. Aby sme to prekonali, môžeme trénovať vlastné vektorové reprezentácie na základe nášho datasetu.


## Kontextové vkladania

Jedným z hlavných obmedzení tradičných predtrénovaných reprezentácií vkladania, ako je Word2Vec, je skutočnosť, že aj keď dokážu zachytiť určitý význam slova, nedokážu rozlíšiť rôzne významy. To môže spôsobiť problémy v následných modeloch.

Napríklad slovo „hrať“ má rôzny význam v týchto dvoch vetách:
- Išiel som na **hru** do divadla.
- John chce **hrať** so svojimi priateľmi.

Predtrénované vkladania, o ktorých sme hovorili, reprezentujú oba významy slova „hrať“ rovnakým spôsobom. Aby sme prekonali toto obmedzenie, potrebujeme vytvoriť vkladania založené na **jazykovom modeli**, ktorý je trénovaný na veľkom korpuse textu a *vie*, ako môžu byť slová spájané v rôznych kontextoch. Diskusia o kontextových vkladaní je mimo rozsah tohto tutoriálu, ale vrátime sa k nim, keď budeme hovoriť o jazykových modeloch v ďalšej jednotke.



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho rodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
